###Team Members = Tomiris, Kulash, Yussuf
***NN Model - ResNet-50***

## Import Library

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

import sys
sys.path.insert(0,'/content/drive/MyDrive/')

Mounted at /content/drive/


In [ ]:
cd /content/drive/MyDrive/finalproject

/content/drive/MyDrive/finalproject


In [ ]:
!pip install mne==0.18.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.3 MB 11.4 MB/s 
  Created wheel for mne: filename=mne-0.18.0-py3-none-any.whl size=6313880 sha256=5b3a94066048111015e475fb87da2a29c089bbdf79d8a69dcd7b3840d70e93b3
  Stored in directory: /root/.cache/pip/wheels/e4/3c/6a/73096375347433a5e5978da35abdec5423a02e34d307924917
Successfully built mne


In [ ]:
from nu_smrutils import loaddat,subject_specific,augment_dataset,crop_data,train_model

import torch 
import itertools
import pandas as pd 
import pickle 
import numpy as np
import mne

from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable 

%matplotlib inline
%load_ext autoreload
%autoreload 2
#%matplotlib qt 
torch.manual_seed(0)

## Necessary functions

In [ ]:
scaler = StandardScaler()

def preprocessing(data):
  sh1=data.shape[1]
  sh2=data.shape[2]
  data=data.reshape(data.shape[0],-1)
  data=scaler.fit_transform(data)
  data=data.reshape(data.shape[0],sh1,sh2)
  return data

In [ ]:
#by default 
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if dev.type == 'cuda':
   print('Your GPU device name :', torch.cuda.get_device_name())      

Your GPU device name : Tesla T4


In [ ]:
from torch.utils.data import TensorDataset, DataLoader

def get_data_loaders(dat, batch_size, EEGNET = None, dev=dev):    
    # convert data dimensions to into to gray scale image format
    if EEGNET: ### EEGNet model requires the last dimension to be 1 
        ff = lambda dat: torch.unsqueeze(dat, dim = -1)    
    else:
        ff = lambda dat: torch.unsqueeze(dat, dim = 1)    
        
    x_train, x_valid, x_test =  map(ff,(dat['xtrain'],dat['xvalid'],dat['xtest']))    
    y_train, y_valid, y_test =  dat['ytrain'], dat['yvalid'], dat['ytest']
    print('Input data shape', x_train.shape)       
    
    # TensorDataset & Dataloader    
    train_dat = TensorDataset(x_train.to(dev), y_train.to(dev)) 
    val_dat   = TensorDataset(x_valid.to(dev), y_valid.to(dev)) 
    
    train_loader = DataLoader(train_dat, batch_size = batch_size, shuffle = True)
    val_loader   = DataLoader(val_dat,   batch_size = batch_size, shuffle = False)

    output = dict(dset_loaders = {'train': train_loader, 'val': val_loader}, 
                  dset_sizes  =  {'train': len(x_train), 'val': len(x_valid)},
                  test_data   =  {'x_test' : x_test, 'y_test' : y_test})          
    return output 

Here we provided architecture defining for ResNet-50,which consist of Bottleneck blocks. We added dropout to avoid overfitting

In [ ]:
class Bottleneck(nn.Module):
    """
Bottleneck for ResNet-50
    Args:
        in_channels (int): number of input channels
        out_channels (int): number of output channels
        stride (int): convolutional stride
        norm (str): type of normalization
    """

    def __init__(self, in_channels, inter_channels, identity_downsample=None, stride=1):
        super(Bottleneck, self).__init__()
        self.expansion=4
        self.in_channels = in_channels
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=inter_channels,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=False,
        )
        self.bn1 = nn.BatchNorm2d(inter_channels)
        self.conv2 = nn.Conv2d(
            in_channels=inter_channels,
            out_channels=inter_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False,
        )
        self.bn2 = nn.BatchNorm2d(inter_channels)
        self.conv3 = nn.Conv2d(
            in_channels=inter_channels,
            out_channels=inter_channels*self.expansion,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=False,
        )
        self.bn3 = nn.BatchNorm2d(inter_channels * self.expansion)
        self.relu = nn.LeakyReLU()
        self.identity_downsample = identity_downsample
        self.stride = stride

    def forward(self, x):
        identity=x.clone()
        out = self.conv1(x)
        out = self.bn1(out)
        #out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.conv3(out)
        out = self.bn3(out)
        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)
        out += identity
        out = self.relu(out)
        return out

In [ ]:
class Resnet_50(nn.Module):

    def __init__(self,cin=1, classes=None):
        super(Resnet_50, self).__init__()
        self.in_channels = 64
        self.conv1=nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3,bias=False)
        self.bn1=nn.BatchNorm2d(64)
        self.r=nn.LeakyReLU()
        self.mp=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
        self.l1=self._make_layer(Bottleneck, 3, inter_channels=64, stride=1)
        self.l2=self._make_layer(Bottleneck, 4, inter_channels=128, stride=2)
        self.l3=self._make_layer(Bottleneck, 6, inter_channels=256, stride=2)
        self.l4=self._make_layer(Bottleneck, 3, inter_channels=512, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(512 * 4, 100)
        self.Dropout = nn.Dropout(0.5)  
        self.fc2 = nn.Linear(100, classes)
        

    def forward(self, x):
      x = self.conv1(x)
      x = self.bn1(x)
      x = self.r(x)
      x = self.mp(x)
      x = self.l1(x)
      x = self.l2(x)
      x = self.l3(x)
      x = self.l4(x)
      x = self.avgpool(x)
      x = x.reshape(x.shape[0], -1)
      x = self.fc1(x)
      x=self.Dropout(x)
      x = self.fc2(x)

      return x

    def _make_layer(self, block, num_residual_blocks, inter_channels, stride):
        identity_downsample = None
        layers = []

        if stride != 1 or self.in_channels != inter_channels * 4:
            identity_downsample = nn.Sequential(
                nn.Conv2d(
                    self.in_channels,
                    inter_channels * 4,
                    kernel_size=1,
                    stride=stride,
                    bias=False
                ),
                nn.BatchNorm2d(inter_channels * 4),
            )

        layers.append(
            block(self.in_channels, inter_channels, identity_downsample, stride)
        )

        # The expansion size is always 4 for ResNet 50
        self.in_channels = inter_channels * 4

        # For example for first resnet layer: 256 will be mapped to 64 as intermediate layer,
        # then finally back to 256. Hence no identity downsample is needed, since stride = 1,
        # and also same amount of channels.
        for i in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels, inter_channels))

        return nn.Sequential(*layers)

## Dataset BNCI2014004


Select Dataset

In [ ]:
dname = dict(BNCI2014004 = 'aBNCI2014004R.pickle',
             BNCI2014001 = 'aBNCI2014001R.pickle',
             Weibo2014   = 'aWeibo2014R.pickle',
             Physionet   = 'aPhysionetRR.pickle')

itemname = 'BNCI2014004'

filename = dname[itemname]
iname = itemname + '__'    

data = loaddat(filename)
print('Number of subjects in data :', len(data))

Number of subjects in data : 9


Load subject specific data and preprocess it

In [ ]:
print('\nfilter')
for i in range(len(data)):
  data[i].filter(l_freq = 8, h_freq = 12)

#--------------------------------------------------------------------------

subjectIndex = list(range(108))
class_name = ['left_hand', 'right_hand']
print('\nsubject_specific')
datt = subject_specific(data, subjectIndex, class_name, test_size = 0.15)


#--------------------------------------------------------------------------
# augment only the training data 
print('\nAugment')
augmdata = dict(std_dev = 0.00001,  multiple = 2)

for ii, dat in enumerate(datt):    
    xtrain, ytrain = augment_dataset(dat['xtrain'], dat['ytrain'], 
                                     std_dev = augmdata['std_dev'], 
                                     multiple = augmdata['multiple'])
    
    print("Shape after data augmentation :", xtrain.shape)
    datt[ii]['xtrain'], datt[ii]['ytrain'] = xtrain, ytrain

#--------------------------------------------------------------------------
# Data Cropping
print('\nCropping')
fs = 80 # sampling frequency 
crop_len = 1 #or None
crop = dict(fs = fs, crop_len = crop_len)

for ii, dat in enumerate(datt):
    if crop['crop_len']:
       X_train, y_train = crop_data(crop['fs'], crop['crop_len'], dat['xtrain'], 
                                    dat['ytrain'],xpercent = 50)
       X_valid, y_valid = crop_data(crop['fs'], crop['crop_len'], dat['xvalid'], 
                                    dat['yvalid'],xpercent = 50)
       X_test,  y_test  = crop_data(crop['fs'], crop['crop_len'], dat['xtest'], 
                                    dat['ytest'], xpercent = 50)
       
       print(X_train.shape)
       datt[ii] = dict(xtrain = X_train, xvalid = X_valid, xtest = X_test,
                       ytrain = y_train, yvalid = y_valid, ytest = y_test) 
                        
#--------------------------------------------------------------------------
# Preprocessing dataset
print('\nPreprocessing')
for i in range(0,len(datt)):
  datt[i]['xtrain']=preprocessing(datt[i]['xtrain'])
  datt[i]['xvalid']=preprocessing(datt[i]['xvalid'])
  datt[i]['xtest']=preprocessing(datt[i]['xtest'])
  datt[i]['xtrain']=torch.tensor(datt[i]['xtrain'])
  datt[i]['xvalid']=torch.tensor(datt[i]['xvalid'])
  datt[i]['xtest']=torch.tensor(datt[i]['xtest'])
  datt[i]['xtrain']=datt[i]['xtrain'].type('torch.FloatTensor')
  datt[i]['xvalid']=datt[i]['xvalid'].type('torch.FloatTensor')
  datt[i]['xtest']=datt[i]['xtest'].type('torch.FloatTensor')  

#--------------------------------------------------------------------------
# Dataloaders
print('\nDataloaders')
for ii, dat in enumerate(datt):  
    print('Subjects :', ii+1)
    datt[ii] = get_data_loaders(dat, batch_size = 64)


filter


/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/filter.py:182: Deprec


subject_specific
Loading subject: 0
Loading subject: 1
Loading subject: 2
Loading subject: 3
Loading subject: 4
Loading subject: 5
Loading subject: 6
Loading subject: 7
Loading subject: 8
Loading subject: 9
list index out of range

Augment
Shape after data augmentation : torch.Size([1040, 3, 321])
Shape after data augmentation : torch.Size([982, 3, 321])
Shape after data augmentation : torch.Size([1040, 3, 321])
Shape after data augmentation : torch.Size([1068, 3, 321])
Shape after data augmentation : torch.Size([1068, 3, 321])
Shape after data augmentation : torch.Size([1040, 3, 321])
Shape after data augmentation : torch.Size([1040, 3, 321])
Shape after data augmentation : torch.Size([1098, 3, 321])
Shape after data augmentation : torch.Size([1040, 3, 321])

Cropping
torch.Size([3120, 3, 120])
torch.Size([2946, 3, 120])
torch.Size([3120, 3, 120])
torch.Size([3204, 3, 120])
torch.Size([3204, 3, 120])
torch.Size([3120, 3, 120])
torch.Size([3120, 3, 120])
torch.Size([3294, 3, 120])
tor

Training loop for dataset 

In [ ]:
#Hyperparameter settings
num_epochs = 100
learning_rate = 1e-4
weight_decay = 1e-4  
batch_size = 64
verbose = 2

fs = 80
# define kernel size in terms of ms length 
time_window = 100 #ms
width = time_window*fs//1000    

# ker = 8 #timelength//chans 
h, w = 3, 1  #hight and width of a rectangular kernel    

#% used to save the results table 
results = {}        
table = pd.DataFrame(columns = ['Train_Acc', 'Val_Acc', 'Test_Acc', 'Epoch']) 
sum=0
num=0
max=0

for i in range(len(datt)): 
  subject = i
  print('----------------Subject '+str(subject+1) + '---------------------')
  description = 'Subject'+str(subject+1)
  dat = datt[subject]

  #% get input size (channel x timepoints)
  input_size = (1, dat['test_data']['x_test'].shape[-2], 
                 dat['test_data']['x_test'].shape[-1])

  model = Resnet_50(cin=1,classes=2)    

  # optimizer and the loss function definition 
  optimizer = torch.optim.Adam(model.parameters(), 
                             lr = learning_rate,
                             weight_decay = weight_decay)
  criterion = torch.nn.CrossEntropyLoss()

  model.to(dev)  
  criterion.to(dev)       

  #print("Model architecture >>>", model)

  #******** Training loop *********    
  best_model, train_losses, val_losses, train_accs, val_accs, info =\
      train_model(model, dat['dset_loaders'], dat['dset_sizes'], 
                  criterion, optimizer, dev=dev, lr_scheduler=False, 
                  num_epochs = num_epochs, verbose = verbose)
          
  
  test_samples = 50
  x_test = dat['test_data']['x_test'][:test_samples,:,:] 
  y_test = dat['test_data']['y_test'][:test_samples] 
  #print(x_test.shape)

  # predict test data 
  preds = best_model(x_test.to(dev)) 
  preds_class = preds.data.max(1)[1]

  # get the accuracy 
  corrects = torch.sum(preds_class == y_test.data.to(dev))     
  test_acc = corrects.cpu().numpy()/x_test.shape[0]
  print("Test Accuracy :", test_acc) 
  sum+=test_acc
  num+=1
  if test_acc>max:
    max=test_acc
  # save results       
  tab = dict(Train_Acc= train_accs[info['best_epoch']],
            Val_Acc  = val_accs[info['best_epoch']],   
            Test_Acc = test_acc, Epoch = info['best_epoch'] + 1)         

  table.loc[description] = tab  
  val_acc = np.max(val_accs)
  print('\n---------------------------------------------------------------------\n')

----------------Subject 1---------------------
Epoch 1/100
train loss: 0.0123, acc: 0.5099
val loss: 0.0124, acc: 0.5145
Epoch 2/100
train loss: 0.0106, acc: 0.6141
val loss: 0.0126, acc: 0.5688
Epoch 3/100
train loss: 0.0079, acc: 0.7397
val loss: 0.0158, acc: 0.5507
Epoch 4/100
train loss: 0.0070, acc: 0.7715
val loss: 0.0206, acc: 0.5725
Epoch 5/100
train loss: 0.0058, acc: 0.8103
val loss: 0.0167, acc: 0.5362
Epoch 6/100
train loss: 0.0054, acc: 0.8160
val loss: 0.0243, acc: 0.5507
Epoch 7/100
train loss: 0.0049, acc: 0.8494
val loss: 0.0304, acc: 0.5217
Epoch 8/100
train loss: 0.0043, acc: 0.8679
val loss: 0.0304, acc: 0.5362
Epoch 9/100
train loss: 0.0041, acc: 0.8702
val loss: 0.0231, acc: 0.6196
Epoch 10/100
train loss: 0.0043, acc: 0.8676
val loss: 0.0315, acc: 0.5507
Epoch 11/100
train loss: 0.0035, acc: 0.8981
val loss: 0.0283, acc: 0.4964
Epoch 12/100
train loss: 0.0032, acc: 0.9109
val loss: 0.0273, acc: 0.5362
Epoch 13/100
train loss: 0.0031, acc: 0.9131
val loss: 0.0318,

In [ ]:
print(table)
results[description] = dict(train_accs = train_accs, val_accs =  val_accs,                                
                            ytrain = info['ytrain'], yval= info['yval'])      

fname = iname + 'CNN_POOLED' + description + '_' + str(val_acc)[:4]
torch.save(best_model.state_dict(), fname) 

print(f"\nAverage between subjects: {(sum/num):.2f}")
print(f"Max test_score between subjects: {max:.2f}")

          Train_Acc   Val_Acc  Test_Acc Epoch
Subject1   0.870192  0.619565      0.56     9
Subject2   0.765445  0.593870      0.46    12
Subject3   0.999359  0.561594      0.38    81
Subject4   0.960362  0.817544      0.84    39
Subject5   0.882022  0.603509      0.48    54
Subject6   0.993269  0.626812      0.62    54
Subject7   0.717949  0.634058      0.60     3
Subject8   0.987857  0.766323      0.68    31
Subject9   0.993590  0.608696      0.56    66

Average between subjects: 0.58
Max test_score between subjects: 0.84


## Dataset BNCI2014001


Select Dataset

In [ ]:
dname = dict(BNCI2014004 = 'aBNCI2014004R.pickle',
             BNCI2014001 = 'aBNCI2014001R.pickle',
             Weibo2014   = 'aWeibo2014R.pickle',
             Physionet   = 'aPhysionetRR.pickle')

itemname = 'BNCI2014001'

filename = dname[itemname]
iname = itemname + '__'    

data = loaddat(filename)
print('Number of subjects in data :', len(data))

Number of subjects in data : 9


Load subject specific data and preprocess it

In [ ]:
print('\nfilter')
for i in range(len(data)):
  data[i].filter(l_freq = 8, h_freq = 12)

#--------------------------------------------------------------------------

subjectIndex = list(range(108))
class_name = ['left_hand', 'right_hand']
print('\nsubject_specific')
datt = subject_specific(data, subjectIndex, class_name, test_size = 0.15)


#--------------------------------------------------------------------------
# augment only the training data 
print('\nAugment')
augmdata = dict(std_dev = 0.000001,  multiple = 2)

for ii, dat in enumerate(datt):    
    xtrain, ytrain = augment_dataset(dat['xtrain'], dat['ytrain'], 
                                     std_dev = augmdata['std_dev'], 
                                     multiple = augmdata['multiple'])
    
    print("Shape after data augmentation :", xtrain.shape)
    datt[ii]['xtrain'], datt[ii]['ytrain'] = xtrain, ytrain

#--------------------------------------------------------------------------
# Data Cropping
print('\nCropping')
fs = 80 # sampling frequency 
crop_len = 1 #or None
crop = dict(fs = fs, crop_len = crop_len)

for ii, dat in enumerate(datt):
    if crop['crop_len']:
       X_train, y_train = crop_data(crop['fs'], crop['crop_len'], dat['xtrain'], 
                                    dat['ytrain'],xpercent = 50)
       X_valid, y_valid = crop_data(crop['fs'], crop['crop_len'], dat['xvalid'], 
                                    dat['yvalid'],xpercent = 50)
       X_test,  y_test  = crop_data(crop['fs'], crop['crop_len'], dat['xtest'], 
                                    dat['ytest'], xpercent = 50)
       
       print(X_train.shape)
       datt[ii] = dict(xtrain = X_train, xvalid = X_valid, xtest = X_test,
                       ytrain = y_train, yvalid = y_valid, ytest = y_test) 
                        
#--------------------------------------------------------------------------
# Preprocessing dataset
print('\nPreprocessing')
for i in range(0,len(datt)):
  datt[i]['xtrain']=preprocessing(datt[i]['xtrain'])
  datt[i]['xvalid']=preprocessing(datt[i]['xvalid'])
  datt[i]['xtest']=preprocessing(datt[i]['xtest'])
  datt[i]['xtrain']=torch.tensor(datt[i]['xtrain'])
  datt[i]['xvalid']=torch.tensor(datt[i]['xvalid'])
  datt[i]['xtest']=torch.tensor(datt[i]['xtest'])
  datt[i]['xtrain']=datt[i]['xtrain'].type('torch.FloatTensor')
  datt[i]['xvalid']=datt[i]['xvalid'].type('torch.FloatTensor')
  datt[i]['xtest']=datt[i]['xtest'].type('torch.FloatTensor')    

#--------------------------------------------------------------------------
# Dataloaders
print('\nDataloaders')
for ii, dat in enumerate(datt):  
    print('Subjects :', ii+1)
    datt[ii] = get_data_loaders(dat, batch_size = 64)


filter


/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/filter.py:182: Deprec


subject_specific
Loading subject: 0
Loading subject: 1
Loading subject: 2
Loading subject: 3
Loading subject: 4
Loading subject: 5
Loading subject: 6
Loading subject: 7
Loading subject: 8
Loading subject: 9
list index out of range

Augment
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])

Cropping
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 

Training loop for dataset 

In [ ]:
#Hyperparameter settings
num_epochs = 100
learning_rate = 1e-3
weight_decay = 1e-3  
batch_size = 64
verbose = 2

fs = 80
# define kernel size in terms of ms length 
time_window = 100 #ms
width = time_window*fs//1000    

# ker = 8 #timelength//chans 
h, w = 3, 1  #hight and width of a rectangular kernel    

#% used to save the results table 
results = {}        
table = pd.DataFrame(columns = ['Train_Acc', 'Val_Acc', 'Test_Acc', 'Epoch']) 
sum=0
num=0
max=0

for i in range(len(datt)): 
  subject = i
  print('----------------Subject '+str(subject+1) + '---------------------')
  description = 'Subject'+str(subject+1)
  dat = datt[subject]

  #% get input size (channel x timepoints)
  input_size = (1, dat['test_data']['x_test'].shape[-2], 
                 dat['test_data']['x_test'].shape[-1])

  model = Resnet_50(cin=1,classes=2)

  # optimizer and the loss function definition 
  optimizer = torch.optim.Adam(model.parameters(), 
                             lr = learning_rate,
                             weight_decay = weight_decay)
  criterion = torch.nn.CrossEntropyLoss()

  model.to(dev)  
  criterion.to(dev)       

  #print("Model architecture >>>", model)

  #******** Training loop *********    
  best_model, train_losses, val_losses, train_accs, val_accs, info =\
      train_model(model, dat['dset_loaders'], dat['dset_sizes'], 
                  criterion, optimizer, dev=dev, lr_scheduler=False, 
                  num_epochs = num_epochs, verbose = verbose)
          
  
  test_samples = 50
  x_test = dat['test_data']['x_test'][:test_samples,:,:] 
  y_test = dat['test_data']['y_test'][:test_samples] 
  #print(x_test.shape)

  # predict test data 
  preds = best_model(x_test.to(dev)) 
  preds_class = preds.data.max(1)[1]

  # get the accuracy 
  corrects = torch.sum(preds_class == y_test.data.to(dev))     
  test_acc = corrects.cpu().numpy()/x_test.shape[0]
  print("Test Accuracy :", test_acc) 
  sum+=test_acc
  num+=1
  if test_acc>max:
    max=test_acc
  # save results       
  tab = dict(Train_Acc= train_accs[info['best_epoch']],
            Val_Acc  = val_accs[info['best_epoch']],   
            Test_Acc = test_acc, Epoch = info['best_epoch'] + 1)         

  table.loc[description] = tab  
  val_acc = np.max(val_accs)
  print('\n---------------------------------------------------------------------\n')

----------------Subject 1---------------------
Epoch 1/100
train loss: 0.0266, acc: 0.5322
val loss: 0.0136, acc: 0.5135
Epoch 2/100
train loss: 0.0142, acc: 0.4968
val loss: 0.0316, acc: 0.4955
Epoch 3/100
train loss: 0.0121, acc: 0.5580
val loss: 0.0131, acc: 0.5315
Epoch 4/100
train loss: 0.0126, acc: 0.5966
val loss: 0.0118, acc: 0.6126
Epoch 5/100
train loss: 0.0103, acc: 0.6425
val loss: 0.0107, acc: 0.6486
Epoch 6/100
train loss: 0.0076, acc: 0.7689
val loss: 0.0108, acc: 0.6757
Epoch 7/100
train loss: 0.0048, acc: 0.8800
val loss: 0.0239, acc: 0.5495
Epoch 8/100
train loss: 0.0042, acc: 0.8953
val loss: 0.0190, acc: 0.6216
Epoch 9/100
train loss: 0.0024, acc: 0.9420
val loss: 0.0154, acc: 0.6126
Epoch 10/100
train loss: 0.0025, acc: 0.9340
val loss: 0.0177, acc: 0.6937
Epoch 11/100
train loss: 0.0018, acc: 0.9573
val loss: 0.0231, acc: 0.6216
Epoch 12/100
train loss: 0.0008, acc: 0.9831
val loss: 0.0201, acc: 0.7117
Epoch 13/100
train loss: 0.0017, acc: 0.9622
val loss: 0.0332,

In [ ]:
print(table)
results[description] = dict(train_accs = train_accs, val_accs =  val_accs,                                
                            ytrain = info['ytrain'], yval= info['yval'])      

fname = iname + 'CNN_POOLED' + description + '_' + str(val_acc)[:4]
torch.save(best_model.state_dict(), fname) 

print(f"\nAverage between subjects: {(sum/num):.2f}")
print(f"Max test_score between subjects: {max:.2f}")

          Train_Acc   Val_Acc  Test_Acc Epoch
Subject1   1.000000  0.747748      0.68    50
Subject2   0.680354  0.594595      0.56    18
Subject3   1.000000  0.918919      0.82    47
Subject4   0.962963  0.648649      0.64    23
Subject5   0.949275  0.540541      0.44    32
Subject6   0.982287  0.666667      0.62    61
Subject7   0.961353  0.594595      0.48    18
Subject8   0.996779  0.891892      0.76    54
Subject9   0.991143  0.837838      0.80    16

Average between subjects: 0.64
Max test_score between subjects: 0.82


## Dataset Weibo2014


Select Dataset

In [ ]:
dname = dict(BNCI2014004 = 'aBNCI2014004R.pickle',
             BNCI2014001 = 'aBNCI2014001R.pickle',
             Weibo2014   = 'aWeibo2014R.pickle',
             Physionet   = 'aPhysionetRR.pickle')

itemname = 'Weibo2014'

filename = dname[itemname]
iname = itemname + '__'    

data = loaddat(filename)
print('Number of subjects in data :', len(data))

Number of subjects in data : 10


Load subject specific data and preprocess it

In [ ]:
print('\nfilter')
for i in range(len(data)):
  data[i].filter(l_freq = 8, h_freq = 12)

#--------------------------------------------------------------------------

subjectIndex = list(range(108))
class_name = ['left_hand', 'right_hand']
print('\nsubject_specific')
datt = subject_specific(data, subjectIndex, class_name, test_size = 0.15)


#--------------------------------------------------------------------------
# augment only the training data 
print('\nAugment')
augmdata = dict(std_dev = 0.00001,  multiple = 2)

for ii, dat in enumerate(datt):    
    xtrain, ytrain = augment_dataset(dat['xtrain'], dat['ytrain'], 
                                     std_dev = augmdata['std_dev'], 
                                     multiple = augmdata['multiple'])
    
    print("Shape after data augmentation :", xtrain.shape)
    datt[ii]['xtrain'], datt[ii]['ytrain'] = xtrain, ytrain

#--------------------------------------------------------------------------
# Data Cropping
print('\nCropping')
fs = 80 # sampling frequency 
crop_len = 1 #or None
crop = dict(fs = fs, crop_len = crop_len)

for ii, dat in enumerate(datt):
    if crop['crop_len']:
       X_train, y_train = crop_data(crop['fs'], crop['crop_len'], dat['xtrain'], 
                                    dat['ytrain'],xpercent = 50)
       X_valid, y_valid = crop_data(crop['fs'], crop['crop_len'], dat['xvalid'], 
                                    dat['yvalid'],xpercent = 50)
       X_test,  y_test  = crop_data(crop['fs'], crop['crop_len'], dat['xtest'], 
                                    dat['ytest'], xpercent = 50)
       
       print(X_train.shape)
       datt[ii] = dict(xtrain = X_train, xvalid = X_valid, xtest = X_test,
                       ytrain = y_train, yvalid = y_valid, ytest = y_test) 
                        
#--------------------------------------------------------------------------
# Preprocessing dataset
print('\nPreprocessing')
for i in range(0,len(datt)):
  datt[i]['xtrain']=preprocessing(datt[i]['xtrain'])
  datt[i]['xvalid']=preprocessing(datt[i]['xvalid'])
  datt[i]['xtest']=preprocessing(datt[i]['xtest'])
  datt[i]['xtrain']=torch.tensor(datt[i]['xtrain'])
  datt[i]['xvalid']=torch.tensor(datt[i]['xvalid'])
  datt[i]['xtest']=torch.tensor(datt[i]['xtest'])
  datt[i]['xtrain']=datt[i]['xtrain'].type('torch.FloatTensor')
  datt[i]['xvalid']=datt[i]['xvalid'].type('torch.FloatTensor')
  datt[i]['xtest']=datt[i]['xtest'].type('torch.FloatTensor')   

#--------------------------------------------------------------------------
# Dataloaders
print('\nDataloaders')
for ii, dat in enumerate(datt):  
    print('Subjects :', ii+1)
    datt[ii] = get_data_loaders(dat, batch_size = 64)


filter


/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/filter.py:182: Deprec


subject_specific
Loading subject: 0
Loading subject: 1
Loading subject: 2
Loading subject: 3
Loading subject: 4
Loading subject: 5
Loading subject: 6
Loading subject: 7
Loading subject: 8
Loading subject: 9
Loading subject: 10
list index out of range

Augment
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([202, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])

Cropping
torch.Size([690, 60, 120])
torch.Size([690, 60, 120])
torch.Size([690, 60, 120])
torch.Size([690, 60, 120])
torch.Size([690, 60, 120])
torc

Training loop for dataset 

In [ ]:
#Hyperparameter settings
num_epochs = 100
learning_rate = 1e-3
weight_decay = 1e-4 
batch_size = 64
verbose = 2

fs = 80
# define kernel size in terms of ms length 
time_window = 100 #ms
width = time_window*fs//1000    

# ker = 8 #timelength//chans 
h, w = 3, 1  #hight and width of a rectangular kernel    

#% used to save the results table 
results = {}        
table = pd.DataFrame(columns = ['Train_Acc', 'Val_Acc', 'Test_Acc', 'Epoch']) 
sum=0
num=0
max=0

for i in range(len(datt)): 
  subject = i
  print('----------------Subject '+str(subject+1) + '---------------------')
  description = 'Subject'+str(subject+1)
  dat = datt[subject]

  #% get input size (channel x timepoints)
  input_size = (1, dat['test_data']['x_test'].shape[-2], 
                 dat['test_data']['x_test'].shape[-1])

  model=Resnet_50(cin=1,classes=2)

  # optimizer and the loss function definition 
  optimizer = torch.optim.Adam(model.parameters(), 
                             lr = learning_rate,
                             weight_decay = weight_decay)
  criterion = torch.nn.CrossEntropyLoss()

  model.to(dev)  
  criterion.to(dev)       

  #print("Model architecture >>>", model)

  #******** Training loop *********    
  best_model, train_losses, val_losses, train_accs, val_accs, info =\
      train_model(model, dat['dset_loaders'], dat['dset_sizes'], 
                  criterion, optimizer, dev=dev, lr_scheduler=False, 
                  num_epochs = num_epochs, verbose = verbose)
          
  
  test_samples = 50
  x_test = dat['test_data']['x_test'][:test_samples,:,:] 
  y_test = dat['test_data']['y_test'][:test_samples] 
  #print(x_test.shape)

  # predict test data 
  preds = best_model(x_test.to(dev)) 
  preds_class = preds.data.max(1)[1]

  # get the accuracy 
  corrects = torch.sum(preds_class == y_test.data.to(dev))     
  test_acc = corrects.cpu().numpy()/x_test.shape[0]
  print("Test Accuracy :", test_acc) 
  sum+=test_acc
  num+=1
  if test_acc>max:
    max=test_acc
  # save results       
  tab = dict(Train_Acc= train_accs[info['best_epoch']],
            Val_Acc  = val_accs[info['best_epoch']],   
            Test_Acc = test_acc, Epoch = info['best_epoch'] + 1)         

  table.loc[description] = tab  
  val_acc = np.max(val_accs)
  print('\n---------------------------------------------------------------------\n')

----------------Subject 1---------------------
Epoch 1/100
train loss: 0.0271, acc: 0.5130
val loss: 0.0878, acc: 0.5079
Epoch 2/100
train loss: 0.0170, acc: 0.4913
val loss: 0.3519, acc: 0.4921
Epoch 3/100
train loss: 0.0145, acc: 0.5101
val loss: 0.3396, acc: 0.5238
Epoch 4/100
train loss: 0.0133, acc: 0.5159
val loss: 0.5379, acc: 0.5238
Epoch 5/100
train loss: 0.0128, acc: 0.5652
val loss: 0.2428, acc: 0.4603
Epoch 6/100
train loss: 0.0120, acc: 0.5725
val loss: 0.0801, acc: 0.4286
Epoch 7/100
train loss: 0.0148, acc: 0.5304
val loss: 0.4635, acc: 0.4921
Epoch 8/100
train loss: 0.0116, acc: 0.5667
val loss: 0.0164, acc: 0.4921
Epoch 9/100
train loss: 0.0108, acc: 0.5551
val loss: 0.1063, acc: 0.5238
Epoch 10/100
train loss: 0.0102, acc: 0.5855
val loss: 0.0803, acc: 0.4444
Epoch 11/100
train loss: 0.0093, acc: 0.6507
val loss: 0.0730, acc: 0.4286
Epoch 12/100
train loss: 0.0083, acc: 0.7101
val loss: 0.1047, acc: 0.4444
Epoch 13/100
train loss: 0.0066, acc: 0.7768
val loss: 0.0347,

In [ ]:
print(table)
results[description] = dict(train_accs = train_accs, val_accs =  val_accs,                                
                            ytrain = info['ytrain'], yval= info['yval'])      

fname = iname + 'CNN_POOLED' + description + '_' + str(val_acc)[:4]
torch.save(best_model.state_dict(), fname) 

print(f"\nAverage between subjects: {(sum/num):.2f}")
print(f"Max test_score between subjects: {max:.2f}")

           Train_Acc   Val_Acc  Test_Acc Epoch
Subject1    0.998551  0.587302      0.40    70
Subject2    1.000000  0.698413      0.56    92
Subject3    0.872464  0.714286      0.52    48
Subject4    0.504348  0.603175      0.46     9
Subject5    0.500000  0.619048      0.44    12
Subject6    0.993399  0.759259      0.56    54
Subject7    0.895652  0.936508      0.80     6
Subject8    0.553623  0.619048      0.40    10
Subject9    0.910145  0.650794      0.60    29
Subject10   0.808696  0.571429      0.40    44

Average between subjects: 0.51
Max test_score between subjects: 0.80


## Dataset Physionet


Select Dataset

In [ ]:
dname = dict(BNCI2014004 = 'aBNCI2014004R.pickle',
             BNCI2014001 = 'aBNCI2014001R.pickle',
             Weibo2014   = 'aWeibo2014R.pickle',
             Physionet   = 'aPhysionetRR.pickle')

itemname = 'Physionet'

filename = dname[itemname]
iname = itemname + '__'    

data = loaddat(filename)
print('Number of subjects in data :', len(data))

Number of subjects in data : 109


Load subject specific data and preprocess it

In [ ]:
print('\nfilter')
for i in range(len(data)):
  data[i].filter(l_freq = 8, h_freq = 12)

#--------------------------------------------------------------------------

subjectIndex = list(range(108))
class_name = ['left_hand', 'right_hand']
print('\nsubject_specific')
datt = subject_specific(data, subjectIndex, class_name, test_size = 0.1)


#--------------------------------------------------------------------------
# augment only the training data 
print('\nAugment')
augmdata = dict(std_dev = 0.001,  multiple = 2)

for ii, dat in enumerate(datt):    
    xtrain, ytrain = augment_dataset(dat['xtrain'], dat['ytrain'], 
                                     std_dev = augmdata['std_dev'], 
                                     multiple = augmdata['multiple'])
    
    print("Shape after data augmentation :", xtrain.shape)
    datt[ii]['xtrain'], datt[ii]['ytrain'] = xtrain, ytrain

#--------------------------------------------------------------------------
# Data Cropping
print('\nCropping')
fs = 80 # sampling frequency 
crop_len = 1 #or None
crop = dict(fs = fs, crop_len = crop_len)

for ii, dat in enumerate(datt):
    if crop['crop_len']:
       X_train, y_train = crop_data(crop['fs'], crop['crop_len'], dat['xtrain'], 
                                    dat['ytrain'],xpercent = 50)
       X_valid, y_valid = crop_data(crop['fs'], crop['crop_len'], dat['xvalid'], 
                                    dat['yvalid'],xpercent = 50)
       X_test,  y_test  = crop_data(crop['fs'], crop['crop_len'], dat['xtest'], 
                                    dat['ytest'], xpercent = 50)
       
       print(X_train.shape)
       datt[ii] = dict(xtrain = X_train, xvalid = X_valid, xtest = X_test,
                       ytrain = y_train, yvalid = y_valid, ytest = y_test) 
                        
#--------------------------------------------------------------------------
# Preprocessing dataset
print('\nPreprocessing')
for i in range(0,len(datt)):
  datt[i]['xtrain']=preprocessing(datt[i]['xtrain'])
  datt[i]['xvalid']=preprocessing(datt[i]['xvalid'])
  datt[i]['xtest']=preprocessing(datt[i]['xtest'])
  datt[i]['xtrain']=torch.tensor(datt[i]['xtrain'])
  datt[i]['xvalid']=torch.tensor(datt[i]['xvalid'])
  datt[i]['xtest']=torch.tensor(datt[i]['xtest'])
  datt[i]['xtrain']=datt[i]['xtrain'].type('torch.FloatTensor')
  datt[i]['xvalid']=datt[i]['xvalid'].type('torch.FloatTensor')
  datt[i]['xtest']=datt[i]['xtest'].type('torch.FloatTensor')   

#--------------------------------------------------------------------------
# Dataloaders
print('\nDataloaders')
for ii, dat in enumerate(datt):  
    print('Subjects :', ii+1)
    datt[ii] = get_data_loaders(dat, batch_size = 64)


filter
Setting up band-pass filter from 8 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 12.00 Hz
- Upper transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 13.50 Hz)
- Filter length: 133 samples (1.663 sec)



/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/filter.py:182: Deprec

Setting up band-pass filter from 8 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 12.00 Hz
- Upper transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 13.50 Hz)
- Filter length: 133 samples (1.663 sec)



/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/filter.py:182: Deprec

Setting up band-pass filter from 8 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 12.00 Hz
- Upper transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 13.50 Hz)
- Filter length: 133 samples (1.663 sec)



...

Setting up band-pass filter from 8 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 12.00 Hz
- Upper transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 13.50 Hz)
- Filter length: 133 samples (1.663 sec)



/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/filter.py:182: Deprec

Setting up band-pass filter from 8 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 12.00 Hz
- Upper transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 13.50 Hz)
- Filter length: 133 samples (1.663 sec)



/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/filter.py:182: Deprec


subject_specific
Loading subject: 0
Loading subject: 1
Loading subject: 2
Loading subject: 3
Loading subject: 4
Loading subject: 5
Loading subject: 6
Loading subject: 7
Loading subject: 8
Loading subject: 9
Loading subject: 10
Loading subject: 11
Loading subject: 12
Loading subject: 13
Loading subject: 14
Loading subject: 15
Loading subject: 16
Loading subject: 17
Loading subject: 18
Loading subject: 19
Loading subject: 20
Loading subject: 21
Loading subject: 22
Loading subject: 23
Loading subject: 24
Loading subject: 25
Loading subject: 26
Loading subject: 27
Loading subject: 28
Loading subject: 29
Loading subject: 30
Loading subject: 31
Loading subject: 32
Loading subject: 33
Loading subject: 34
Loading subject: 35
Loading subject: 36
Loading subject: 37
Loading subject: 38
Loading subject: 39
Loading subject: 40
Loading subject: 41
Loading subject: 42
Loading subject: 43
Loading subject: 44
Loading subject: 45
Loading subject: 46
Loading subject: 47
Loading subject: 48
Loading subj

Training loop for dataset 

In [ ]:
#Hyperparameter settings
num_epochs = 80
learning_rate = 1e-4
weight_decay = 1e-4
batch_size = 64
verbose = 2

fs = 80
# define kernel size in terms of ms length 
time_window = 100 #ms
width = time_window*fs//1000    

# ker = 8 #timelength//chans 
h, w = 3, 1  #hight and width of a rectangular kernel    

#% used to save the results table 
results = {}        
table = pd.DataFrame(columns = ['Train_Acc', 'Val_Acc', 'Test_Acc', 'Epoch']) 
sum=0
num=0
max=0

for i in range(len(datt)): 
  subject = i
  print('----------------Subject '+str(subject+1) + '---------------------')
  description = 'Subject'+str(subject+1)
  dat = datt[subject]

  #% get input size (channel x timepoints)
  input_size = (1, dat['test_data']['x_test'].shape[-2], 
                 dat['test_data']['x_test'].shape[-1])

  model = Resnet_50(cin=1,classes=2)

  # optimizer and the loss function definition 
  optimizer = torch.optim.Adam(model.parameters(), 
                             lr = learning_rate,
                             weight_decay = weight_decay)
  criterion = torch.nn.CrossEntropyLoss()

  model.to(dev)  
  criterion.to(dev)       

  #print("Model architecture >>>", model)

  #******** Training loop *********    
  best_model, train_losses, val_losses, train_accs, val_accs, info =\
      train_model(model, dat['dset_loaders'], dat['dset_sizes'], 
                  criterion, optimizer, dev=dev, lr_scheduler=False, 
                  num_epochs = num_epochs, verbose = verbose)
          
  
  test_samples = 50
  x_test = dat['test_data']['x_test'][:test_samples,:,:] 
  y_test = dat['test_data']['y_test'][:test_samples] 
  #print(x_test.shape)

  # predict test data 
  preds = best_model(x_test.to(dev)) 
  preds_class = preds.data.max(1)[1]

  # get the accuracy 
  corrects = torch.sum(preds_class == y_test.data.to(dev))     
  test_acc = corrects.cpu().numpy()/x_test.shape[0]
  print("Test Accuracy :", test_acc) 
  sum+=test_acc
  num+=1
  if test_acc>max:
    max=test_acc
  # save results       
  tab = dict(Train_Acc= train_accs[info['best_epoch']],
            Val_Acc  = val_accs[info['best_epoch']],   
            Test_Acc = test_acc, Epoch = info['best_epoch'] + 1)         

  table.loc[description] = tab  
  val_acc = np.max(val_accs)
  print('\n---------------------------------------------------------------------\n')

Выходные данные были обрезаны до нескольких последних строк (5000).
Epoch 70/80
train loss: 0.0002, acc: 1.0000
val loss: 0.1153, acc: 0.5625
Epoch 71/80
train loss: 0.0001, acc: 1.0000
val loss: 0.1162, acc: 0.5000
Epoch 72/80
train loss: 0.0023, acc: 0.9651
val loss: 0.1270, acc: 0.5000
Epoch 73/80
train loss: 0.0009, acc: 0.9826
val loss: 0.1294, acc: 0.5000
Epoch 74/80
train loss: 0.0026, acc: 0.9535
val loss: 0.0885, acc: 0.6250
Epoch 75/80
train loss: 0.0038, acc: 0.9244
val loss: 0.1169, acc: 0.5625
Epoch 76/80
train loss: 0.0012, acc: 0.9767
val loss: 0.1424, acc: 0.5000
Epoch 77/80
train loss: 0.0011, acc: 0.9593
val loss: 0.1685, acc: 0.5000
Epoch 78/80
train loss: 0.0004, acc: 1.0000
val loss: 0.2193, acc: 0.5000
Epoch 79/80
train loss: 0.0001, acc: 1.0000
val loss: 0.2571, acc: 0.5000
Epoch 80/80
train loss: 0.0001, acc: 1.0000
val loss: 0.2683, acc: 0.5000
Training complete in 0m 29s
Best val Acc: 0.750000
Best Epoch : 25
Test Accuracy : 0.3333333333333333

---------------

In [ ]:
print(table)
results[description] = dict(train_accs = train_accs, val_accs =  val_accs,                                
                            ytrain = info['ytrain'], yval= info['yval'])      

fname = iname + 'CNN_POOLED' + description + '_' + str(val_acc)[:4]
torch.save(best_model.state_dict(), fname) 

print(f"\nAverage between subjects: {(sum/num):.2f}")
print(f"Max test_score between subjects: {max:.2f}")

            Train_Acc   Val_Acc  Test_Acc Epoch
Subject1     0.750000  0.750000       0.3    33
Subject2     0.806452  0.833333       0.7    12
Subject3     0.867647  0.750000       0.5    70
Subject4     0.943548  0.750000       0.4    77
Subject5     0.741935  0.666667       0.1    11
...               ...       ...       ...   ...
Subject104   0.791667  0.583333       0.5    20
Subject105   0.758065  0.666667       0.4     8
Subject106   0.786765  0.833333       0.9    33
Subject107   0.801471  0.583333       0.8    36
Subject108   1.000000  0.833333       0.4    42

[108 rows x 4 columns]

Average between subjects: 0.52
Max test_score between subjects: 0.90


In [ ]:
table[:]

,Train_Acc,Val_Acc,Test_Acc,Epoch
Subject1,0.750000,0.750000,0.3,33
Subject2,0.806452,0.833333,0.7,12
Subject3,0.867647,0.750000,0.5,70
Subject4,0.943548,0.750000,0.4,77
Subject5,0.741935,0.666667,0.1,11
...,...,...,...,...
Subject104,0.791667,0.583333,0.5,20
Subject105,0.758065,0.666667,0.4,8
Subject106,0.786765,0.833333,0.9,33
Subject107,0.801471,0.583333,0.8,36
